In [1]:
import os

In [2]:
%pwd

'd:\\Research_Project\\MyWork\\MLOPS\\Projects\\Chest_Cancer_Classification_using_mlflow_dvc-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Research_Project\\MyWork\\MLOPS\\Projects\\Chest_Cancer_Classification_using_mlflow_dvc-'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.cnnClassifier.constant import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config
    
    

In [8]:
import os
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size


In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file path: str
        extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

    
except Exception as e:
    raise e

[2024-09-26 15:22:25,183: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-26 15:22:25,185: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-26 15:22:25,186: INFO: common: created directory at: artifacts]
[2024-09-26 15:22:25,187: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-26 15:22:25,188: INFO: 1090528670: Downloading data from https://drive.google.com/file/d/1NBSfCz28cBsglgCnxM2WI1wyAmzdSdie/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1NBSfCz28cBsglgCnxM2WI1wyAmzdSdie
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1NBSfCz28cBsglgCnxM2WI1wyAmzdSdie&confirm=t&uuid=304f5676-2d71-4e1d-9f07-247b86482d40
To: d:\Research_Project\MyWork\MLOPS\Projects\Chest_Cancer_Classification_using_mlflow_dvc-\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:05<00:00, 9.47MB/s]

[2024-09-26 15:22:35,652: INFO: 1090528670: Download data from https://drive.google.com/file/d/1NBSfCz28cBsglgCnxM2WI1wyAmzdSdie/view?usp=sharing into file artifacts/data_ingestion/data.zip]
